In [ ]:
!conda install -yq PyMuPDF

In [1]:
import datetime
import dotenv
import os
import time

import boto3
import fitz
import requests

dotenv.load_dotenv()

API_KEY = os.getenv("EDINET_API_KEY")
MODEL_ID = os.getenv("BEDROCK_MODEL_ID")
BEDROCK_CLIENT = boto3.client("bedrock-runtime", region_name="ap-northeast-1")

In [2]:
def tool_use_for_seccode():
    return {"toolSpec": {"name": "for_seccode",
                         "description": "正しい会社名を推測して、その会社の数字4桁の証券コードを返すツール",
                         "inputSchema": {"json": {"type": "object",
                                                  "properties": {"company_name": {"type": "string",
                                                                                  "description": "プロンプト内に記載されている会社名"},
                                                                 "predict_company_name": {"type": "string",
                                                                                          "description": "推測した正確な会社名"},
                                                                 "company_seccode": {"type": "string",
                                                                                     "description": "数字4桁の証券コード",
                                                                                     "pattern": "^[0-9]{4}$"}},
                                                  "required": ["company_name", "predict_company_name", "company_seccode"]}}}}  # 単なるstructured outputのみでのツール定義の場合
                                                  # "required": ["company_name"]}}}}  # MCPクライアント/サーバを実装しているツール定義の場合


def call_claude_for_seccode(bedrock_client, user_prompt):
    SYSTEM_PROMPT = """
    以下のユーザプロンプトには会社名と思われるものが記載されています。そこから正確な会社名を推測して、その会社の証券コード(数字4桁)を出力してください。

    ユーザプロンプト : {a}
    """.format(a=user_prompt)
    claude_input_message = [{"content": [{"text": SYSTEM_PROMPT}],
                             "role": "user"}]
    response = bedrock_client.converse(modelId=MODEL_ID,
                                       messages=claude_input_message,
                                       inferenceConfig={"temperature": 0.2},
                                       toolConfig={"tools": [tool_use_for_seccode()],
                                                   "toolChoice": {"tool": {"name": "for_seccode"}}})
    response_company_name = response["output"]["message"]["content"][0]["toolUse"]["input"]["company_name"]
    response_predict_company_name = response["output"]["message"]["content"][0]["toolUse"]["input"]["predict_company_name"]
    response_company_seccode = response["output"]["message"]["content"][0]["toolUse"]["input"]["company_seccode"]
    return response_company_name, response_predict_company_name, response_company_seccode

In [3]:
def search_target_doc_id(api_key, target_date, security_code):
    URL = "https://disclosure.edinet-fsa.go.jp/api/v2/documents.json"
    PARAM = {"date": target_date, 
             "type": 2,
             "Subscription-Key": api_key}
    response = requests.request(method="GET",
                                url=URL,
                                params=PARAM)
    response.raise_for_status()  # 呼び出し元の関数のtry-except構文でキャッチ出来るようにstatus code 200以外は例外とする
    response_json = response.json()
    # print(response_json)
    target_security_code = "{a}0".format(a=security_code)  # EDINET向けに証券コード4桁を右に0でパディングする形で5桁にする
    # target_form_code = "030000"  # 有価証券報告書
    target_form_code = "043A00"  # 半期報告書
    for result_dict in response_json["results"]:
        if (result_dict["secCode"] == target_security_code) and (result_dict["formCode"] == target_form_code):
            return result_dict["docID"]
    raise Exception()


def get_pdf_as_object(api_key, doc_id):
    URL = "https://disclosure.edinet-fsa.go.jp/api/v2/documents/{a}".format(a=doc_id)
    PARAM = {"type": 2,
             "Subscription-Key": api_key}
    response = requests.request(method="GET",
                                url=URL,
                                params=PARAM)
    response.raise_for_status()
    return response.content


def extract_text_from_pdf_object(obj):
    text = ""
    with fitz.open(stream=obj, filetype="pdf") as doc:
        for page_num, page_doc in enumerate(doc, start=1):
            text = text + "\n *** page number {a} *** \n".format(a=page_num)
            text = text + page_doc.get_text()
    return text

In [4]:
def call_claude_for_summarized_report(bedrock_client, data):
    SYSTEM_PROMPT = """
    あなたは経験豊富な株式投資家です。
    以下の半期報告書から、株式投資で重要と思われる情報を抽出してください。

    半期報告書 : {a}
    """.format(a=data)
    claude_input_message = [{"role": "user",
                             "content": [{"text": SYSTEM_PROMPT}]}]
    response = bedrock_client.converse(modelId=MODEL_ID,
                                       messages=claude_input_message,
                                       inferenceConfig={"temperature": 0.2})
    return response

In [5]:
def main(api_key, bedrock_client, user_prompt):
    name, predict_name, seccode = call_claude_for_seccode(bedrock_client=bedrock_client,
                                                          user_prompt=user_prompt)
    print(name, predict_name, seccode)
    today = datetime.datetime.today()
    for i in range(365):
        target_date = (today - datetime.timedelta(days=i)).strftime("%Y-%m-%d")
        try:
            target_doc_id = search_target_doc_id(api_key=API_KEY,
                                                 target_date=target_date,
                                                 security_code=seccode)
            break
        except Exception as e:
            time.sleep(1)
            # print("{a}：対象の有価証券報告書はありませんでした。".format(a=target_date))
            continue
    target_pdf_object = get_pdf_as_object(api_key=API_KEY,
                                          doc_id=target_doc_id)
    target_text = extract_text_from_pdf_object(obj=target_pdf_object)
    print(len(target_text))
    res = call_claude_for_summarized_report(bedrock_client=bedrock_client,
                                            data=target_text)
    res_text = res["output"]["message"]["content"][0]["text"]
    print(res_text)

In [6]:
main(api_key=API_KEY,
     bedrock_client=BEDROCK_CLIENT,
     user_prompt="pkshaの有価証券報告書を調べて")

pksha PKSHAテクノロジー 3993
29988
株式投資の観点から、以下の重要な情報を抽出しました：

業績・財務状況：
- 売上収益：10,072百万円(前年同期比24.0%増)
- 税引前中間利益：3,310百万円(前年同期比48.6%増) 
- 親会社の所有者に帰属する中間利益：2,131百万円(前年同期比51.8%増)

セグメント情報：
1. AI Research & Solution事業
- 売上収益：5,910百万円(前年同期比23.8%増)
- セグメント利益：1,287百万円(前年同期比23.3%増)

2. AI SaaS事業 
- 売上収益：4,238百万円(前年同期比23.6%増)
- セグメント利益：1,530百万円(前年同期比2.3%増)

注目すべきポイント：
- 深刻化する人材不足とAIの技術進化による顧客ニーズの高まりを背景に、両事業とも成長
- AI Research & Solution事業ではソリューション案件数が増加
- AI SaaS事業ではプロダクトの導入社数および年間経常収益が積み上がっている
- 子会社Sapeetが東証グロース市場に上場し、持分法適用関連会社に変更

財務指標：
- 親会社所有者帰属持分比率：74.4%
- 基本的1株当たり中間利益：68.67円
- 希薄化後1株当たり中間利益：68.60円

投資判断において、AIビジネスの成長性と収益性の向上が顕著であり、特に注目に値します。
